In [1]:
##DEPENDENCIES
import pandas as pd
import datetime
from datetime import timedelta, date
import yfinance as yf
import numpy as np
import yahoofinancials
from yahoofinancials import YahooFinancials

In [2]:
##LIST OF DATES
#define function daterange
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
dates = []
#defne start and end date
#note: end date is day after last day of data
start_date = date(2019, 1, 1)
end_date = date(2019, 4, 1)
#loop through all dates, adding each to list
for single_date in daterange(start_date, end_date):
    curr_date = single_date.strftime("%Y-%m-%d")
    dates.append(curr_date)    

In [3]:
##SCRAPE YAHOO FOR EARNINGS DATA

#creating empty data frame for raw data
column_names = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)', 'Earnings Date']
combined_df = pd.DataFrame(columns = column_names)

#define start and end date
start = start_date.strftime("%Y-%m-%d")
end = end_date.strftime("%Y-%m-%d")
#loop through each day
i=0
while i < len(dates):
    day = dates[i]
    url = 'https://finance.yahoo.com/calendar/earnings?from='+start+'&to='+end+'&day='+day
    try:
        tables = pd.read_html(url)
    except:
        i=i+1
    else:
        df = tables[0]
        df.columns = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']
        j=0
        earn_dates = []
        while j < len(df['Symbol']):
            earn_dates.append(day)
            j=j+1
        df['Earnings Date'] = earn_dates
        combined_df = combined_df.append(df)
        i=i+1    
#raw data sent to .csv file
#note: rename output file for current quarter
combined_df = combined_df.sort_values(by=['Symbol', 'Earnings Date'])
combined_df.to_csv('resources/2019q1_raw_earnings.csv')
combined_df.head()

,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),Earnings Date
9,A,Agilent Technologies Inc,TAS,0.73,0.76,4.11,2019-02-20
0,AA,Alcoa Corp,TAS,0.50,0.66,32.00,2019-01-16
21,AABA,Altaba Inc,Before Market Open,NaN,NaN,NaN,2019-02-27
3,AAL,American Airlines Group Inc,TAS,1.01,1.04,2.97,2019-01-24
9,AAMC,Altisource Asset Management Corp,TAS,NaN,NaN,NaN,2019-02-27


In [4]:
##Scrape wikipedia for S&P 500 symbols
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
sp_symbols = list(df['Symbol'])
sp_symbols = sorted(sp_symbols)

In [5]:
##CREATE DICTIONARIES FOR EACH S&P 500 STOCK
#define lists
earnings_dates = []
multiples = []
sym_row = []
raw_symbols = list(combined_df['Symbol'])
unique_raw = []
for x in raw_symbols:
    if x not in unique_raw: 
        unique_raw.append(x)
#loop through S&P 500 symbols        
for sym in sp_symbols:
    #if no data for symbol
    if sym not in raw_symbols:
        earn_dict = {'Symbol' : sym,
                    '2019Q1 Time' : 'NaN',
                    '2019Q1 Date' : 'NaN'}  
    #if data for symbol    
    else:
        sym_row = [sym]
        for row in combined_df.iterrows():
            if row[1]['Symbol'] == sym:
                sym_row[0] = sym
                sym_row.append(row[1]['Earnings Call Time'])
                sym_row.append(row[1]['Earnings Date'])
        #if symbol has multiple dates        
        if len(sym_row) > 3:
            earn_dict = {'Symbol' : sym,
                    '2019Q1 Time' : 'Multi',
                    '2019Q1 Date' : 'Multi'}
            #create list of symbols with multiple dates (with the dates included)
            multiples.append(sym_row)
        #if symbol has only one date    
        else:
            earn_dict = {'Symbol' : sym_row[0],
                        '2019Q1 Time' : sym_row[1],
                        '2019Q1 Date' : sym_row[2]}        
    earnings_dates.append(earn_dict)
earnings_dates

[{'Symbol': 'A', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-02-20'},
 {'Symbol': 'AAL', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-24'},
 {'Symbol': 'AAP', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-02-19'},
 {'Symbol': 'AAPL', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-29'},
 {'Symbol': 'ABBV', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-25'},
 {'Symbol': 'ABC', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-31'},
 {'Symbol': 'ABMD', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-31'},
 {'Symbol': 'ABT', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-23'},
 {'Symbol': 'ACN', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-03-28'},
 {'Symbol': 'ADBE', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-03-14'},
 {'Symbol': 'ADI', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-02-20'},
 {'Symbol': 'ADM', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-02-05'},
 {'Symbol': 'ADP', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-01-30'},
 {'Symbol': 'ADS', '2019Q1 Time': 'TAS', '2019Q1 Date': '2019-02-07'},
 {'S

In [6]:
#S&P 500 stocks with multiple earnings dates in same quarter
mdf = pd.DataFrame(multiples)
mdf.to_csv('resources/2019q1_sp_multi_earnings.csv')
mdf

,0,1,2,3,4,5,6,7,8
0,CI,Before Market Open,2019-02-01,Time Not Supplied,2019-02-01,None,None,None,None
1,EVRG,TAS,2019-02-12,TAS,2019-02-22,None,None,None,None
2,INFO,TAS,2019-01-15,TAS,2019-03-26,None,None,None,None
3,LEN,TAS,2019-01-09,TAS,2019-03-27,None,None,None,None
4,MKC,TAS,2019-01-24,TAS,2019-03-26,None,None,None,None
5,PBCT,TAS,2019-01-17,TAS,2019-01-17,None,None,None,None
6,PGR,Before Market Open,2019-01-23,Before Market Open,2019-02-13,Before Market Open,2019-02-27,Before Market Open,2019-03-13
7,ROL,Before Market Open,2019-01-22,TAS,2019-01-23,None,None,None,None
8,WRK,Before Market Open,2019-01-28,Before Market Open,2019-01-31,None,None,None,None
9,ZION,After Market Close,2019-01-21,After Market Close,2019-01-22,None,None,None,None


In [7]:
#Data frame for all S&P 500 stocks
sp_quarter = pd.DataFrame(earnings_dates)
sp_quarter = sp_quarter[['Symbol', '2019Q1 Time', '2019Q1 Date']]
sp_quarter.to_csv('resources/2019q1_sp_earnings.csv')
sp_quarter.head()

,Symbol,2019Q1 Time,2019Q1 Date
0,A,TAS,2019-02-20
1,AAL,TAS,2019-01-24
2,AAP,TAS,2019-02-19
3,AAPL,TAS,2019-01-29
4,ABBV,TAS,2019-01-25
